In [1]:
# calculate five evaluation metrics
def calculate_metrics(df):
    # true positive and true negative
    subdf = df[df.predict == df.obs] # true prediction
    TP = len(subdf[subdf.predict == 1].index) # true positive
    TN = len(subdf[subdf.predict == 0].index) # true negative

    # false positive and false negative
    subdf = df[df.predict != df.obs] # false prediction 
    FN = len(subdf[subdf.predict == 0].index)# false negative
    FP = len(subdf[subdf.predict == 1].index) # false positive

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)

    sensitivity = TP/(TP+FN)
    specificity = TN/(TN+FP)
    balanced_accuracy = (sensitivity+specificity)/2
    accuracy = (TP+TN)/(TP+TN+FP+FN)

    f1 =  0 if precision + recall == 0 else 2 * precision * recall / (precision + recall)
    out = pd.DataFrame(data = {'precision': [precision], 
                               'recall':[recall], 
                               'f1':[f1],
                               'sensitivity':[sensitivity],
                               'specificity':[specificity],
                               'balanced_accuracy':[balanced_accuracy], 
                               'accuracy':[accuracy]})
    return out


# for RS CNN alone
th = 0.1 # using 10 cm threshold

dir_aso = '/data0/kyang33/data/ASO/ori/ASO_3M_SD_USCOGE_20180524.tif' # Swiss site only has binary data
shp = gpd.read_file('/data0/kyang33/project/pc2_snow_cover_mapping_meadows/pipeline/data/results/validation/GIS/USCOGE_20180524_extent_remove_clouds.shp')
ds_aso = xr.open_rasterio(dir_aso).rio.clip(shp.geometry,from_disk=True)


dir_pred = dir_out + '/USCOGE_20180524_ndvi/'
dir_RS = '/data0/kyang33/project/pc2_snow_cover_mapping_meadows/pipeline/data/SCA/CNN_AJI_CO/'
fn_pred = glob.glob(dir_pred + '*SCA.tif')

# used_list = pd.read_csv('/data0/kyang33/project/pc2_snow_cover_mapping_meadows/pipeline/data/SCA/USCOGE_20180524_ndvi/used_sence.txt', header = None)
# fn_pred = [f for f in fn_pred if os.path.basename(f).split('_SCA')[0] in list(used_list.iloc[:,0])]

validation_res = pd.DataFrame()
num = []
files = []
method = []
for i in range(len(fn_pred)):
    ds_planet = xr.open_rasterio(fn_pred[i])
    ds_aso_clip = ds_aso.rio.reproject_match(ds_planet)
    
    # fine CNN-SCA 
    f_cnn = glob.glob(dir_RS + os.path.basename(fn_pred[i]).split('_SCA')[0] + '*.tif')
    if len(f_cnn) == 1:
        ds_cnn = xr.open_rasterio(f_cnn[0])
        print(f_cnn[0])
        ds_cnn_clip = ds_cnn.rio.reproject_match(ds_planet)
    
    
   
        df = pd.DataFrame()
        df['ASO_SD'] = np.reshape(ds_aso_clip.values, (np.product(ds_aso_clip.values.shape),))
        df['obs'] = np.where(df['ASO_SD'] > th, 1.0, 0.0)
        df['predict'] = np.reshape(ds_cnn_clip.values, (np.product(ds_cnn_clip.values.shape),))


        df = df[df['ASO_SD'] != -9999]
        df = df[df['predict'] != 9999]
        
        if df['obs'].mean() > 0: # for images having snow 

            print("overall model performance:")
            print(calculate_metrics(df))
            validation_res=pd.concat([validation_res, calculate_metrics(df)])

            method.append('CNN')
            num.append(df.shape[0])
            files.append(f_cnn[0])

validation_res['th'] = th
validation_res['file_SCA'] = files
validation_res['file_ASO'] = dir_aso
validation_res['pixel_num'] = num
validation_res['method'] = method

validation_res['Image_ID'] = [f.split('/')[-1].split('_')[1] for f in validation_res.file_SCA]

NameError: name 'gpd' is not defined